# Clustering and Segmenting Toronto Neighborhoods

In [46]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
# soup.prettify()

## Scrape wikipedia page for table

In [47]:
my_table = soup.find('table',{'class':'wikitable sortable'})

In [48]:
table_rows = my_table.findAll('tr')

In [49]:
header = ['PostalCode', 'Borough', 'Neighborhood']
header

['PostalCode', 'Borough', 'Neighborhood']

In [50]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
neighborhoods = pd.DataFrame(l, columns=header)

In [51]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


## Clean up data

In [52]:
# Remove newlines in Neighborhood column
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace('\n', '')
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [60]:
# Change all the unknowns to a common value
neighborhoods = neighborhoods.replace('Not assigned', None)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,None,None
2,M2A,None,None
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [61]:
# Assign all unknown Neighborhood's with the Borough Value
neighborhoods['Neighborhood'].fillna(neighborhoods['Borough'], inplace=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,None,None
2,M2A,None,None
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [62]:
# Drop all rows with unassigned Borough
neighborhoods.dropna(how='any', axis='index', inplace=True)
neighborhoods.head(5)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [63]:
# Group by Postal Code
neighborhoods = neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Output Shape & Rows

In [65]:
neighborhoods.shape

(178, 3)

In [66]:
neighborhoods

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


# Get Postal Code Coordinates

In [68]:
!pip install --upgrade geocoder
import geocoder

In [78]:
def coordinate(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return pd.series(latitude, longitude)

In [79]:
head = neighborhoods.head()
head

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
head[['Latitude', 'Longitude']] = neighborhoods.apply(lambda row: coordinate(neighborhoods['PostalCode']), axis=1)

In [ ]:
head